In [ ]:
import sys
sys.path.append('/home/SharedFolder/git/tillvolkmann/night-drive')
sys.path.append('/home/SharedFolder/git/tillvolkmann/night-drive/classifier')
sys.path.append('/home/SharedFolder/git/tillvolkmann/night-drive/utils')

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from eval_utils import *
from classification_eval import *
from weather_classifier import *
from easyimages.utils import make_notebook_wider
%matplotlib inline
make_notebook_wider()

In [ ]:
############################ MODIFY HERE ############################
# set data paths
path_valid_images = "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/train_dev_A"
path_valid_json = "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/annotations/bdd100k_sorted_train_dev_A_over.json"
path_weights = "/home/SharedFolder/trained_models/night-drive/weather_classifier/without_cloudy/train_A_over/resnet18_weather_classifierbdd100k_sorted_train_A_over_epoch_32.pth"

# title for charts
chart_title = "train_A_over -> train_dev_A_over"

# cut_off
cut_off = None

In [ ]:
# setting device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# create classifer, datasets and dataloader
net, _, dl_valid, class_dict = weather_classifier(path_valid_json = path_valid_json, path_valid_images = path_valid_images)
# send model to device
net.to(device)
# load weights
net.load_state_dict(torch.load(path_weights)["model_state_dict"])

In [ ]:
# f1-score
class_dict_cm = class_dict.copy()
if cut_off is not None:
    class_dict_cm["unknown"] = -1 # catching the "negatives" when defining a cut-off

scores, values = evaluate_weather(net, dl_valid, score_types = ["f1_score_micro", "f1_score_macro", "f1_score_weighted", "accuracy", "roc_auc_micro", "roc_auc_macro", "pr_micro", "pr_macro"], cut_off = cut_off, class_dict = class_dict)    
print(scores)

In [ ]:
plot_confusion_matrix(values["targets"], values["predictions"], classes = sorted(class_dict_cm, key = class_dict_cm.get), title = f"Confusion Matrix: {chart_title} (Thresh = {cut_off})", normalize = True)

## Warning: ROC not applicable to imbalanced datasets ! ##

In [ ]:
roc_auc, fpr, tpr, n_classes = calculate_roc_auc(values["targets"], values["prediction_scores"], class_dict)
fig = plot_roc_curve(roc_auc, fpr, tpr, n_classes, class_dict, title = f"ROC: {chart_title}")

## Precision-Recall Curve ##

In [ ]:
precision, recall, average_precision, n_classes = calculate_precision_recall(values["targets"], values["prediction_scores"], class_dict)

In [ ]:
fig = plot_pr_curve(precision, recall, average_precision, n_classes, class_dict, title = f"PR: {chart_title}")

In [ ]:
df_pred = pd.DataFrame(values)
df_pred["max_score"] = df_pred["prediction_scores"].apply(lambda x: np.max(x))
df_pred.head()

In [ ]:
print(f"Number of unique images: {len(df_pred.paths.unique())}")
df_pred.head()

In [ ]:
df_pred_goods = df_pred.loc[df_pred.targets == df_pred.predictions].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
print(f"Number of unique correctly classified images: {len(df_pred_goods.paths.unique())}")
df_pred_goods.head()

In [ ]:
df_pred_bads = df_pred.loc[df_pred.targets != df_pred.predictions].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
print(f"Number of unique misclassified images: {len(df_pred_bads.paths.unique())}")
df_pred_bads.head()

### Clear -> Clear (Best) ###

In [ ]:
df_pred_good_clear = df_pred_goods.loc[df_pred_goods.targets == 0].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
df_pred_good_clear.head()
plot_imagegrid(df_pred_good_clear.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_good_clear.max_score.tolist()])

### Clear -> Clear (Worst) ###

In [ ]:
df_pred_good_clear = df_pred_goods.loc[df_pred_goods.targets == 0].sort_values(by = "max_score", ascending = True).reset_index(drop = True)
df_pred_good_clear.head()
plot_imagegrid(df_pred_good_clear.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_good_clear.max_score.tolist()])

### Rainy -> Rainy (Best) ###

In [ ]:
df_pred_good_rainy = df_pred_goods.loc[df_pred_goods.targets == 1].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
df_pred_good_rainy.head()
plot_imagegrid(df_pred_good_rainy.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_good_rainy.max_score.tolist()])

### Rainy -> Rainy (Worst) ###

In [ ]:
df_pred_good_rainy = df_pred_goods.loc[df_pred_goods.targets == 1].sort_values(by = "max_score", ascending = True).reset_index(drop = True)
df_pred_good_rainy.head()
plot_imagegrid(df_pred_good_rainy.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_good_rainy.max_score.tolist()])

### Snowy -> Snowy (Best) ###

In [ ]:
df_pred_good_snowy = df_pred_goods.loc[df_pred_goods.targets == 2].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
df_pred_good_snowy.head()
plot_imagegrid(df_pred_good_snowy.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_good_snowy.max_score.tolist()])

### Snowy -> Snowy (Worst) ###

In [ ]:
df_pred_good_snowy = df_pred_goods.loc[df_pred_goods.targets == 2].sort_values(by = "max_score", ascending = True).reset_index(drop = True)
df_pred_good_snowy.head()
plot_imagegrid(df_pred_good_snowy.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_good_snowy.max_score.tolist()])

### Clear -> Rainy (Worst) ###

In [ ]:
df_pred_bads_clear = df_pred_bads.loc[df_pred_bads.targets == 0].loc[df_pred_bads.predictions == 1].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
df_pred_bads_clear.head()
plot_imagegrid(df_pred_bads_clear.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_bads_clear.max_score.tolist()])

### Clear -> Snowy (Worst) ###

In [ ]:
df_pred_bads_clear = df_pred_bads.loc[df_pred_bads.targets == 0].loc[df_pred_bads.predictions == 2].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
df_pred_bads_clear.head()
plot_imagegrid(df_pred_bads_clear.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_bads_clear.max_score.tolist()])

### Rainy -> Clear (Worst) ###

In [ ]:
df_pred_bads_rainy = df_pred_bads.loc[df_pred_bads.targets == 1].loc[df_pred_bads.predictions == 0].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
df_pred_bads_rainy.head()
plot_imagegrid(df_pred_bads_rainy.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_bads_rainy.max_score.tolist()])

### Rainy -> Snowy (Worst) ###

In [ ]:
df_pred_bads_rainy = df_pred_bads.loc[df_pred_bads.targets == 1].loc[df_pred_bads.predictions == 2].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
df_pred_bads_rainy.head()
plot_imagegrid(df_pred_bads_rainy.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_bads_rainy.max_score.tolist()])

### Snowy -> Clear (Worst) ###

In [ ]:
df_pred_bads_snowy = df_pred_bads.loc[df_pred_bads.targets == 2].loc[df_pred_bads.predictions == 0].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
df_pred_bads_snowy.head()
plot_imagegrid(df_pred_bads_snowy.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_bads_snowy.max_score.tolist()])

### Snowy -> Rainy (Worst) ###

In [ ]:
df_pred_bads_snowy = df_pred_bads.loc[df_pred_bads.targets == 2].loc[df_pred_bads.predictions == 1].sort_values(by = "max_score", ascending = False).reset_index(drop = True)
df_pred_bads_snowy.head()
plot_imagegrid(df_pred_bads_snowy.paths.tolist(), save_name = None, do_shuffle = False, labels = [f"Score: {score:.5f}" for score in df_pred_bads_snowy.max_score.tolist()])